

#install required libraries, models, dependancies



In [32]:
!pip install -q -U google-generativeai
!pip install patool
!pip install PyMuPDF
!pip install python-docx
!pip install pytesseract
!pip install Pillow

import pytesseract
!apt-get install tesseract-ocr
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

import pathlib
import textwrap
import json
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

model = genai.GenerativeModel('gemini-1.5-flash')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


#you can import data from your local machine through a zip file from here


In [ ]:
from google.colab import files
uploading = files.upload()

Saving resume_data.zip to resume_data.zip


#unzip the zip file

In [ ]:
import patoolib
patoolib.extract_archive("/content/resume_data.zip")

INFO patool: Extracting /content/resume_data.zip ...
INFO:patool:Extracting /content/resume_data.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_3_sdos6u -- /content/resume_data.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_3_sdos6u -- /content/resume_data.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/resume_data.zip extracted to `data'.
INFO:patool:... /content/resume_data.zip extracted to `data'.


'data'

#obtain the gemini api key from google ai.google.dev website

In [11]:
genai.configure(api_key="")

#get response

In [40]:
import os
import re
import fitz  # PyMuPDF
import docx
from PIL import Image
import pytesseract
from difflib import SequenceMatcher

def convert_image_to_text(image_path):
    """Convert image to text using Tesseract OCR."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def convert_pdf_to_text(pdf_path):
    """Convert PDF to text using PyMuPDF and Tesseract OCR for images."""
    pdf_document = fitz.open(pdf_path)
    text = ""

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        # Try to extract text using PyMuPDF
        page_text = page.get_text()
        if page_text.strip():
            text += page_text
        else:
            # Fall back to OCR if text extraction fails
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            page_text = pytesseract.image_to_string(img)
            text += page_text

    return text

def convert_docx_to_text(docx_path):
    """Convert DOCX to text using python-docx."""
    doc = docx.Document(docx_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text



def extract_details_from_text(text):
    prompt = (
        f"Please extract the following details from the given resume text in json format:\n\n"
        f"{text}\n\n"
        """
        remember keep all the dates in dd/mm/yyyy format, give only one primary contact number, also tell only the city and state in location:\n
        Format your response exactly as follows:\n
        {
          "name": "",
          "mobileNumber": "",
          "emailId": "",
          "dob": "",
          "summary": "",
          "location": "",
          "skills": [],
          "education": [
            {
              "education": "",
              "instituteName": "",
              "courseName": "",
              "startYear": "",
              "endYear": ""
            }
          ],
          "experience": [
            {
              "companyName": "",
              "jobTitle": "",
              "startDate": "",
              "endDate": ""
            }
          ]
        }
        print every key or entry of json on new line, keep it readable
        """

    )
    response = model.generate_content(prompt)
    return response

def process_resume_file(file_path):

    filename = os.path.basename(file_path)

    if filename.endswith('.pdf'):
        resume_text = convert_pdf_to_text(file_path)
    elif filename.endswith('.docx'):
        resume_text = convert_docx_to_text(file_path)
    elif filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        resume_text = convert_image_to_text(file_path)
    else:
        print(f"Unsupported file format: {filename}")

    resume_details = extract_details_from_text(resume_text)
    # to_markdown(resume_details.text)
    print(resume_details.text)


# Open a file in write mode
    with open('output.json', 'w') as json_file:
        # Write the data to the JSON file
        json.dump(resume_details.text, json_file)
    # print(resume_details.text)
        # "Using this json schema:\n"



#submit the path to your resume file here
##json data will be printed as well as output.json file will be created

In [41]:
# Example usage
file_path = '/content/new_data/_resume..CV..neuu.docx'
process_resume_file(file_path)

{
  "name": "Ms. Sanchi Suryasen Jadhav",
  "mobileNumber": "8356820997",
  "emailId": "sanchijadhav898@gmail.com",
  "dob": "16/10/1999",
  "summary": "I will always try to use my skills honestly, devotion towards my punctuality etc.\nI will discuss my idealogy with my superiors.",
  "location": "Mumbai, Maharashtra",
  "skills": [
    "Cooking",
    "Baking",
    "Dancing"
  ],
  "education": [
    {
      "education": "S.S.C",
      "instituteName": "St. Joseph's Convent High School",
      "courseName": "",
      "startYear": "",
      "endYear": ""
    },
    {
      "education": "H.S.C",
      "instituteName": "Siddharth College of Arts and Commerce Anand Bhavan",
      "courseName": "",
      "startYear": "",
      "endYear": ""
    },
    {
      "education": "B.Sc in Hospitality Studies",
      "instituteName": "G.D. Ambekar Pratishthan College of Hotel Management",
      "courseName": "",
      "startYear": "",
      "endYear": ""
    }
  ],
  "experience": [
    {
      "com

#give the path to your folder to process multiple resumes
###api issues may occur

In [ ]:
def process_resume_folder(folder_path):
    """Process all resumes in various formats in a folder."""
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            resume_text = convert_pdf_to_text(file_path)
        elif filename.endswith('.docx'):
            resume_text = convert_docx_to_text(file_path)
        elif filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            resume_text = convert_image_to_text(file_path)
        else:
            print(f"Unsupported file format: {filename}")
            continue

        resume_details = extract_details_from_text(resume_text)
        to_markdown(resume_details.text)
        print(resume_details.text)


# Example usage
folder_path = '/content/new_data'
process_resume_folder(folder_path)